In [ ]:
import cv2
import numpy as np
import pyniryo
from pyniryo2 import Vision, NiryoRos

# Define the positions for the Tic Tac Toe field and turn indicators
positions = {
    "p1": (247, 160),
    "p2": (317, 151),
    "p3": (381, 149),
    "p4": (249, 223),
    "p5": (319, 216),
    "p6": (319, 218),
    "p7": (249, 293),
    "p8": (318, 291),
    "p9": (398, 288)
}

indications = {
    "I1": (472, 151),  # Indication point for player 1
    "I2": (468, 266)   # Indication point for player 2
}

# HSV color ranges for red, green, and blue
color_ranges = {
    'red': ([160, 100, 100], [180, 255, 255]),  # Lower and upper bounds for red
    'green': ([40, 50, 50], [80, 255, 255]),    # Lower and upper bounds for green
    'blue': ([100, 150, 0], [140, 255, 255])    # Lower and upper bounds for blue
}

# Colors for drawing rectangles
draw_colors = {
    'red': (0, 0, 255),
    'green': (0, 255, 0),
    'blue': (255, 0, 0)
}

# Initialize the Niryo robot's ROS instance and vision system
ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

# Function to detect color in a given position
def detect_color(hsv_frame, pos, region_size=20):
    x, y = pos
    # Define the region of interest (ROI) around the pixel
    x_start = max(0, x - region_size)
    y_start = max(0, y - region_size)
    x_end = min(hsv_frame.shape[1], x + region_size)
    y_end = min(hsv_frame.shape[0], y + region_size)
    
    region = hsv_frame[y_start:y_end, x_start:x_end]
    
    # Calculate the mean color of the region
    mean_color = cv2.mean(region)[:3]  # Get the average color in the region (ignore alpha channel)
    mean_color = np.array(mean_color, dtype=np.uint8)

    # Check if the mean color falls within any predefined color ranges
    for color_name, (lower, upper) in color_ranges.items():
        lower_bound = np.array(lower, dtype=np.uint8)
        upper_bound = np.array(upper, dtype=np.uint8)

        # Check if the mean color is within the color range
        if all(lower_bound <= mean_color) and all(mean_color <= upper_bound):
            return color_name  # Return the detected color
    return "none"  # Return 'none' if no color matches

# Function to overlay a color mask for detected chips
def overlay_color_mask(frame, pos, color_name, region_size=10):
    x, y = pos
    # Define the region of interest for drawing the rectangle
    x_start = max(0, x - region_size)
    y_start = max(0, y - region_size)
    x_end = min(frame.shape[1], x + region_size)
    y_end = min(frame.shape[0], y + region_size)

    # Get the drawing color (RGB) based on detected color name
    color = draw_colors.get(color_name, (255, 255, 255))  # Default to white if color is 'none'

    # Draw a filled rectangle over the position to indicate the detected color
    cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), color, thickness=-1)

# Initialize a state dictionary to store current colors of each position
state = {pos_name: "none" for pos_name in positions.keys()}

while True:
    # Capture the image from the Niryo robot camera
    img_compressed = vision.get_img_compressed()
    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    # Resize the frame for display and processing
    frame = cv2.resize(img, (640, 480))

    # Convert the image to HSV for better color detection
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Loop through each Tic Tac Toe field position and check for color
    for pos_name, pos in positions.items():
        detected_color = detect_color(hsv_frame, pos)

        if detected_color != state[pos_name]:  # Check if the color has changed
            print(f"State change at {pos_name}: {state[pos_name]} -> {detected_color}")  # Log the state change
            state[pos_name] = detected_color  # Update the state

        if detected_color != "none":
            # Overlay a color mask on the position with the detected color
            overlay_color_mask(frame, pos, detected_color)

        # Display the detected color as text at the position
        cv2.putText(frame, f"{pos_name}: {detected_color}", pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Mark the indication points (Player turn indicators)
    for ind_name, ind_pos in indications.items():
        cv2.putText(frame, ind_name, ind_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Display the frame with color masks and detected colors
    cv2.imshow('Tic Tac Toe Color Detection', frame)

    # Check if 'q' key has been pressed to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup resources
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import pyniryo
from pyniryo2 import Vision, NiryoRos

# Define the positions for the Tic Tac Toe field and turn indicators
positions = {
    "p1": (257, 146),
    "p2": (327, 139),
    "p3": (400, 139),
    "p4": (255, 209),
    "p5": (330, 206),
    "p6": (402, 206),
    "p7": (255, 280),
    "p8": (330, 280),
    "p9": (405, 279),
    "I1": (472, 151),  # Treating I1 as a position
    "I2": (470, 263)   # Treating I2 as a position
}

# Broadened HSV color ranges for red, green, and blue
color_ranges = {
    'red': ([0, 50, 50], [10, 255, 255]),   # Expanded lower and upper bounds for red
    'green': ([35, 40, 40], [85, 255, 255]), # Expanded lower and upper bounds for green
    'blue': ([90, 50, 50], [130, 255, 255])  # Expanded lower and upper bounds for blue
}

# Colors for drawing rectangles
draw_colors = {
    'red': (0, 0, 255),
    'green': (0, 255, 0),
    'blue': (255, 0, 0)
}

# Initialize the Niryo robot's ROS instance and vision system
ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

# Function to apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to enhance contrast
def apply_clahe(image):
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L channel (lightness)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Merge the channels back and convert to BGR
    lab_clahe = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# Function to detect color in a given position
def detect_color(hsv_frame, pos, region_size=15):
    x, y = pos
    # Define the region of interest (ROI) around the pixel
    x_start = max(0, x - region_size)
    y_start = max(0, y - region_size)
    x_end = min(hsv_frame.shape[1], x + region_size)
    y_end = min(hsv_frame.shape[0], y + region_size)
    
    region = hsv_frame[y_start:y_end, x_start:x_end]
    
    # Calculate the mean color of the region
    mean_color = cv2.mean(region)[:3]  # Get the average color in the region (ignore alpha channel)
    mean_color = np.array(mean_color, dtype=np.uint8)

    # Check if the mean color falls within any predefined color ranges
    for color_name, (lower, upper) in color_ranges.items():
        lower_bound = np.array(lower, dtype=np.uint8)
        upper_bound = np.array(upper, dtype=np.uint8)

        # Check if the mean color is within the color range
        if all(lower_bound <= mean_color) and all(mean_color <= upper_bound):
            return color_name  # Return the detected color
    return "none"  # Return 'none' if no color matches

# Function to overlay a color mask for detected chips
def overlay_color_mask(frame, pos, color_name, region_size=10):
    x, y = pos
    # Define the region of interest for drawing the rectangle
    x_start = max(0, x - region_size)
    y_start = max(0, y - region_size)
    x_end = min(frame.shape[1], x + region_size)
    y_end = min(frame.shape[0], y + region_size)

    # Get the drawing color (RGB) based on detected color name
    color = draw_colors.get(color_name, (255, 255, 255))  # Default to white if color is 'none'

    # Draw a filled rectangle over the position to indicate the detected color
    cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), color, thickness=-1)

# Initialize a state dictionary to store current colors of each position
state = {pos_name: "none" for pos_name in positions.keys()}

while True:
    # Capture the image from the Niryo robot camera
    img_compressed = vision.get_img_compressed()
    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    # Resize the frame for display and processing
    frame = cv2.resize(img, (640, 480))

    # Apply Gaussian Blur to reduce noise
    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # Apply CLAHE to enhance contrast
    enhanced_frame = apply_clahe(blurred_frame)

    # Convert the enhanced image to HSV for better color detection
    hsv_frame = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2HSV)

    # Loop through each Tic Tac Toe field position and check for color
    for pos_name, pos in positions.items():
        detected_color = detect_color(hsv_frame, pos)

        if detected_color != state[pos_name]:  # Check if the color has changed
            state[pos_name] = detected_color  # Update the state

            blue_positions = [name for name, color in state.items() if color == 'blue']
            if blue_positions:  # Check if there are any positions with blue
                print(f"Positions with blue: {blue_positions}")

        if detected_color != "none":
            # Overlay a color mask on the position with the detected color
            overlay_color_mask(frame, pos, detected_color)

        # Display the detected color as text at the position
        cv2.putText(frame, f"{pos_name}: {detected_color}", pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the frame with color masks and detected colors
    cv2.imshow('Tic Tac Toe Color Detection', frame)

    # Check if 'q' key has been pressed to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup resources
cv2.destroyAllWindows()
